In [56]:
import pandas as pd
import timeit

In [3]:
df = pd.read_csv('C:\\Users\\johnb\\OneDrive\\Documents\\MSA\\Fall 3\\Machine Learning\\MLProjectData.csv')

In [4]:
df.head()

,num1,num2,num3,num4,num5,num6,num7,num8,num9,num10,...,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,target
0,1424.241,15.70,-1.930008,-0.005399,-0.014426,-0.017944,0.011322,15.22,13.77,13.96,...,False,True,False,False,False,False,False,False,False,21.639999
1,983.787,15.59,0.190003,-0.000129,0.002850,0.017414,0.033371,15.15,15.54,16.64,...,False,False,False,False,False,False,False,False,False,19.919998
2,105.075,16.32,0.809998,0.000752,0.004340,0.025135,0.007954,16.23,16.71,17.44,...,False,False,False,False,False,False,False,False,False,21.150001
3,356.126,28.29,-1.409996,-0.011557,-0.020185,0.003352,-0.047518,26.66,26.24,28.18,...,False,False,False,False,False,False,False,False,False,19.360001
4,772.041,11.49,0.079987,0.001576,0.000724,0.025519,0.033136,11.76,11.97,10.74,...,False,False,False,False,False,False,False,False,False,21.400009


In [5]:
for col in df.columns:
    if col[0:3] == 'cat':
        print(df[col].value_counts())

C    1303
B    1303
D    1279
E    1271
A    1194
Name: cat1, dtype: int64
F    557
H    554
J    550
E    550
C    547
G    529
L    529
D    527
K    510
A    509
I    508
B    480
Name: cat2, dtype: int64
False    6131
True      219
Name: cat3, dtype: int64
False    6070
True      280
Name: cat4, dtype: int64
False    6099
True      251
Name: cat5, dtype: int64
False    6156
True      194
Name: cat6, dtype: int64
False    6192
True      158
Name: cat7, dtype: int64
False    6281
True       69
Name: cat8, dtype: int64
False    6322
True       28
Name: cat9, dtype: int64
False    6306
True       44
Name: cat10, dtype: int64
False    6131
True      219
Name: cat11, dtype: int64
False    6070
True      280
Name: cat12, dtype: int64
False    6099
True      251
Name: cat13, dtype: int64
False    6156
True      194
Name: cat14, dtype: int64
False    6306
True       44
Name: cat15, dtype: int64
False    6322
True       28
Name: cat16, dtype: int64
False    6192
True      158
Name: cat17, dt

In [6]:
dummies1 = pd.get_dummies(df['cat1'], prefix='cat1')
dummies1.head()

,cat1_A,cat1_B,cat1_C,cat1_D,cat1_E
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,0,1
3,0,0,1,0,0
4,0,0,0,0,1


In [7]:
dummies2 = pd.get_dummies(df['cat2'], prefix='cat2')
dummies2.head()

,cat2_A,cat2_B,cat2_C,cat2_D,cat2_E,cat2_F,cat2_G,cat2_H,cat2_I,cat2_J,cat2_K,cat2_L
0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,0,0


In [8]:
df_final = df.join(dummies1).join(dummies2)
df_final.drop(['cat1','cat2'], axis=1, inplace=True)

In [9]:
df_final.describe()

,num1,num2,num3,num4,num5,num6,num7,num8,num9,num10,...,cat2_C,cat2_D,cat2_E,cat2_F,cat2_G,cat2_H,cat2_I,cat2_J,cat2_K,cat2_L
count,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,...,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000,6350.000000
mean,849.859489,19.424687,0.035514,0.000350,0.000692,0.002359,0.004674,19.423997,19.423383,19.422728,...,0.086142,0.082992,0.086614,0.087717,0.083307,0.087244,0.080000,0.086614,0.080315,0.083307
std,1000.217417,8.141230,1.378046,0.011589,0.015842,0.026975,0.036410,8.141693,8.142129,8.142565,...,0.280595,0.275892,0.281291,0.282904,0.276368,0.282215,0.271315,0.281291,0.271802,0.276368
min,0.052000,9.140000,-15.120010,-0.098448,-0.133555,-0.237463,-0.270464,9.140000,9.140000,9.140000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,68.644250,13.420000,-0.529998,-0.004574,-0.006676,-0.010854,-0.012989,13.420000,13.420000,13.420000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,558.720000,17.540000,0.079990,0.000598,0.001327,0.004094,0.007847,17.540000,17.540000,17.540000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1250.553750,23.010000,0.639999,0.005726,0.008740,0.017024,0.025125,23.010000,23.010000,23.010000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8710.263000,80.860000,9.529999,0.145198,0.128249,0.173543,0.205891,80.860000,80.860000,80.860000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
var = df_final.drop('target',axis=1)
target = df_final['target']

In [64]:
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, train_test_split

In [12]:
baseline = [target.mean()] * len(target)
base_score = mean_absolute_error(target, baseline)

In [13]:
reg = linear_model.LinearRegression()
reg.fit(var, target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [14]:
reg.score(var, target)

0.02855167165747352

In [15]:
preds = reg.predict(var)
OLS_mae = mean_absolute_error(target, preds)
base_score - OLS_mae

0.0008437798104576899

In [16]:
lasso = linear_model.Lasso()
lasso.fit(var, target)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [17]:
lasso_preds = lasso.predict(var)
lasso_mae = mean_absolute_error(target, lasso_preds)
base_score - lasso_mae

0.0015274611158285767

In [18]:
from sklearn.tree import DecisionTreeRegressor

In [19]:
tree = DecisionTreeRegressor()
cv = cross_validate(tree, var, target, scoring='neg_mean_absolute_error', return_train_score=True, cv=5)

In [20]:
cv.keys()

dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])

In [21]:
cv['test_score']

array([-1.468004  , -1.41615268, -1.41121311, -1.46245574, -1.38853182])

In [22]:
cv['train_score']

array([-4.25590551e-07, -7.58038058e-07, -7.39304462e-07, -4.58825459e-07,
       -3.84383202e-07])

In [23]:
tree_params = {'max_depth': [i+1 for i in range(10)], 'min_samples_split' :[2,5,10,20], 'min_samples_leaf':[1,2,5,8,10,20,30,40]}
grid = GridSearchCV(tree, param_grid=tree_params,scoring='neg_mean_absolute_error', cv=5)
grid.fit(var, target)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2, 5, 10, 20], 'min_samples_leaf': [1, 2, 5, 8, 10, 20, 30, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [24]:
grid_df = pd.DataFrame(grid.cv_results_)
base_score - (-1 * grid_df['mean_test_score'].max())


C:\Users\johnb\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\johnb\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\johnb\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\johnb\Anaconda3\lib\site-packages\sklearn\util

0.001829154947492806

In [36]:
from sklearn.ensemble import RandomForestRegressor

In [87]:
var_train, var_test, target_train, target_test = train_test_split(var, target, test_size=0.3, random_state=69)
baseline = [target_train.mean()] * len(target_test)
base_score = mean_absolute_error(target_test, baseline)
base_score

0.9602879158667785

In [91]:
start = timeit.default_timer()
test_forest = RandomForestRegressor(n_estimators=500, n_jobs=6, random_state=69)
test_forest.fit(var_train, target_train)
stop = timeit.default_timer()

print('time: ',stop-start)

time:  62.301276814585435


In [92]:
test_forest_preds = test_forest.predict(var_test)
test_forest_mae = mean_absolute_error(target_test, test_forest_preds)
base_score - test_forest_mae

-0.018224755261102565

In [69]:
start = timeit.default_timer()
cv_forest = cross_validate(test_forest, var, target, scoring=['neg_mean_absolute_error','neg_mean_squared_error'], return_train_score=True, cv=5)
stop = timeit.default_timer()

print('time: ',stop-start)

time:  369.8960190282014


In [77]:
print(cv_forest['train_neg_mean_absolute_error'])
print(cv_forest['train_neg_mean_squared_error'])

[-0.36292071 -0.36276081 -0.36489346 -0.36403172 -0.364787  ]
[-0.29044786 -0.29130447 -0.29886637 -0.29534025 -0.30701109]


In [78]:
print(cv_forest['test_neg_mean_absolute_error'])
print(cv_forest['test_neg_mean_squared_error'])

[-0.98300467 -0.98199264 -0.95950743 -0.99118755 -0.98190623]
[-2.18072445 -2.23154712 -1.9485578  -2.2361245  -1.95170937]


In [93]:
forest = RandomForestRegressor(random_state=69)
forest_params = {'n_estimators':[10,100,1000], 'max_features':['auto','log2',10]}

In [97]:
forest_search = GridSearchCV(forest, param_grid=forest_params, scoring=['neg_mean_absolute_error','neg_mean_squared_error'], cv=5, n_jobs=6, refit='neg_mean_absolute_error')

In [98]:
start = timeit.default_timer()
forest_search.fit(var, target)
stop = timeit.default_timer()

print('time: ', stop-start)

time:  903.3776962496559


In [99]:
rand_df = pd.DataFrame(forest_search.cv_results_)


C:\Users\johnb\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_neg_mean_absolute_error'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\johnb\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_neg_mean_absolute_error'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\johnb\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_neg_mean_absolute_error'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C

In [102]:
base_score - (-1 * rand_df['mean_test_neg_mean_absolute_error'].max())

-0.009982061413150611

In [104]:
rand_df[rand_df['mean_test_neg_mean_absolute_error'] == rand_df['mean_test_neg_mean_absolute_error'].max()]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_neg_mean_absolute_error,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,...,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,split0_train_neg_mean_squared_error,split1_train_neg_mean_squared_error,split2_train_neg_mean_squared_error,split3_train_neg_mean_squared_error,split4_train_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_train_neg_mean_squared_error
5,53.939423,1.07597,5.885934,0.261703,log2,1000,"{'max_features': 'log2', 'n_estimators': 1000}",-0.980622,-0.967729,-0.952132,...,-2.067634,0.119914,1,-0.276363,-0.277391,-0.287194,-0.277478,-0.287935,-0.281272,0.005158


In [106]:
import featuretools as ft

In [108]:
es = ft.EntitySet(id='data')
es = es.entity_from_dataframe(entity_id='data',
                             dataframe=var, 
                              index='index',
                             make_index=True)

In [111]:
es['data'].variables

[<Variable: index (dtype = index)>,
 <Variable: num1 (dtype = numeric)>,
 <Variable: num2 (dtype = numeric)>,
 <Variable: num3 (dtype = numeric)>,
 <Variable: num4 (dtype = numeric)>,
 <Variable: num5 (dtype = numeric)>,
 <Variable: num6 (dtype = numeric)>,
 <Variable: num7 (dtype = numeric)>,
 <Variable: num8 (dtype = numeric)>,
 <Variable: num9 (dtype = numeric)>,
 <Variable: num10 (dtype = numeric)>,
 <Variable: num11 (dtype = numeric)>,
 <Variable: num12 (dtype = numeric)>,
 <Variable: num13 (dtype = numeric)>,
 <Variable: num14 (dtype = numeric)>,
 <Variable: num15 (dtype = numeric)>,
 <Variable: num16 (dtype = numeric)>,
 <Variable: num17 (dtype = numeric)>,
 <Variable: num18 (dtype = numeric)>,
 <Variable: num19 (dtype = numeric)>,
 <Variable: num20 (dtype = numeric)>,
 <Variable: num21 (dtype = numeric)>,
 <Variable: num22 (dtype = numeric)>,
 <Variable: num23 (dtype = numeric)>,
 <Variable: num24 (dtype = numeric)>,
 <Variable: num25 (dtype = numeric)>,
 <Variable: num26 (dtyp

In [116]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                     target_entity='data',
                                     trans_primitives=['multiply'])

In [118]:
len(feature_defs)

2950

In [119]:
feature_matrix.head()

,num1,num2,num3,num4,num5,num6,num7,num8,num9,num10,...,num54 * num8,num30 * num29,num57 * num4,num9 * num10,num2 * num48,cat1_E * num32,num9 * num14,num2 * cat2_A,num54 * num3,num59 * num46
index,,,,,,,,,,,,,,,,,,,,,
0,1424.241,15.70,-1.930008,-0.005399,-0.014426,-0.017944,0.011322,15.22,13.77,13.96,...,-1488.785248,-4539.650056,0.626860,192.2292,-2132.544680,0.953682,1.716318,0.0,188.788925,-26345.629051
1,983.787,15.59,0.190003,-0.000129,0.002850,0.017414,0.033371,15.15,15.54,16.64,...,1612.544544,2273.847201,-0.014570,258.5856,1832.365721,0.000000,-0.980589,0.0,20.223650,10760.660228
2,105.075,16.32,0.809998,0.000752,0.004340,0.025135,0.007954,16.23,16.71,17.44,...,-204.446282,1658.742470,-0.015650,291.4224,-687.810908,0.689539,-1.073119,0.0,-10.203394,-1083.022980
3,356.126,28.29,-1.409996,-0.011557,-0.020185,0.003352,-0.047518,26.66,26.24,28.18,...,-1204.215906,-197.305266,0.636324,739.4432,-2941.763208,0.000000,0.102427,0.0,63.688658,-6305.244441
4,772.041,11.49,0.079987,0.001576,0.000724,0.025519,0.033136,11.76,11.97,10.74,...,-499.263942,4856.845688,-0.082459,128.5578,-608.104256,0.951441,0.835894,0.0,-3.395801,-11051.323899


In [ ]:
start = timeit.default_timer()
eng_forest = RandomForestRegressor(n_estimators=100, n_jobs=6, random_state=69)
cv_eng_forest = cross_validate(eng_forest, feature_matrix, target, scoring=['neg_mean_absolute_error','neg_mean_squared_error'], return_train_score=True, cv=5)
stop = timeit.default_timer()

print('time: ',stop-start)